# SEIRHVD Data Fit

This Jupyter notebooks implements the SEIRHVD4 model for fitting the actual Chilean data in order to make projections in bed usage, amount of infected and deaths. 
This implements a single instance to facilitate understanding the model prior to run multiple data fittings at a time.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import linalg as LA

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')
sys.path.insert(1, 'src/SEIRHVD/')
sys.path.insert(1, 'src/utils/')

from class_SEIRHUVD4 import SEIRHVD 
from Quarantine import Quarantine
from importdata import ImportData

Windows


## Time Variables
Declare Initial date for simulating and data fitting

In [124]:
initdate = datetime(2020,5,15)
currentdate = datetime.now()
currentday = (currentdate - initdate).days

## Import Data
To import data we use an ImportData object which is initialized with the region's cut and the initial date. Meanwhile we will adjust this data for regions, then I'll add Heath sector as the minimum territorial unit for SEIRHVD models.

In [125]:
tstate = '13'

# Import Data
RM = ImportData(tstate=tstate,initdate = initdate)
RM.importdata()

Importing General Data
Importing Population
Importing Active infected
Importing Accumulated Infected
Importing Daily Infected
Importing Sochimi Data
Importing Accumulated Deaths
Importing Active Infected by Minciencia
Done


## Simulation Parameters

In [216]:
# Simulation parameters
tsim = 1000
beta = 0.28
mu = 0.8
k = 0.1

SeroPrevFactor=0.035 # SeroPrevalence Factor (The proportion of the population that participates in the dynamics)
expinfection=0 # Proportion in which the exposed infect - 0: nothing, 1: equally as Infected

## Quarantines
Build Quarantine Object:
 
     Q = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once')
     alpha = Q.alpha
     
     Plot Quarantine dynamics:
         Q.plot()

In [217]:
# Quarantines 
max_mob = 0.6
rem_mob = 0.4

#alpha = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once').alpha(t)
alpha = Quarantine(rem_mob).alpha

## Simulation
Initialize Simulation Object, and run the simulation. 

In [218]:
simulation = SEIRHVD(tsim,beta,mu,alpha,k=k,SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,InitialConditions = RM)
simulation.integr_sci(0,tsim,0.1)

InitialCondition Object Data


  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 3349
     njev: 58
      nlu: 58
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 2.60563275e-08, 5.21126551e-08, 2.60615388e-04,
       5.21178663e-04, 7.81741939e-04, 3.38737469e-03, 5.99300745e-03,
       8.59864020e-03, 1.12042730e-02, 3.72606005e-02, 6.33169280e-02,
       8.93732556e-02, 1.15429583e-01, 1.41485911e-01, 4.02049186e-01,
       6.62612461e-01, 9.23175737e-01, 1.18373901e+00, 1.44430229e+00,
       1.70486556e+00, 2.15294047e+00, 2.60101537e+00, 3.04909028e+00,
       3.49716518e+00, 3.94524008e+00, 4.48176417e+00, 5.01828825e+00,
       5.55481233e+00, 6.09133641e+00, 6.60195869e+00, 7.11258096e+00,
       7.62320323e+00, 9.01235885e+00, 1.04015145e+01, 1.17906701e+01,
       1.21379590e+01, 1.24852479e+01, 1.25026124e+01, 1.25199768e+01,
       1.25547057e+01, 1.25633879e+01, 1.25648771e+01, 1.25663663e+01,
       1.25693447e+01, 1.25718241e+01

# Analysis

The following plots will show the simulation results with the real data

In [149]:
# Days to plot (Today + ..)
days = currentday+20
index = np.searchsorted(simulation.t,days)

## Detectable Active Infected
Here we define a Detected infected population as follows:  

 Idet = Imi*det + Ise + Icr  

where det is the proportion of mild detections


**Aun me falta corregir el valor inicial, pues este lo considera para todos los infectados**

In [214]:
det = 0.5
Idet = det*simulation.Imi + simulation.Ise + simulation.Icr

In [144]:
tr_index = np.searchsorted(RM.tr,days)

plt.scatter(RM.tr,RM.Ir,label='Real Data')

plt.xlim(0,days)
plt.plot(simulation.t[:index],Idet[:index],label='Infected')
plt.legend(loc=0)
plt.title('Detectable Active Infected')
plt.show()


## Total Active Infected

In [137]:
tr_index = np.searchsorted(RM.tr,days)

plt.scatter(RM.tr,RM.Ir,label='Real Data')

plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I[:index],label='Total Active Infected')
plt.legend(loc=0)
plt.title('Total Active Infected')
plt.show()


In [27]:
simulation.Imi[0]+simulation.Ias[0]+simulation.Ise[0]+simulation.Icr[0]


12642.000000000002

In [28]:
simulation.I0*simulation.pE_Ise 

421.48428

In [29]:
simulation.Ise[0]

421.48428

In [30]:
RM.Ir[0]

12642

## Daily Infected

In [219]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_d[:index],label='Total Daily new Infected')
plt.scatter(RM.I_d_r_tr,RM.I_d_r,label='Real Data')
plt.legend(loc=0)
plt.show()

In [ ]:
det=0.5


In [153]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],det*simulation.Imi_d[:index]+simulation.Ise_d[:index]+simulation.Icr_d[:index],label='Total Daily new Infected')
plt.scatter(RM.I_d_r_tr,RM.I_d_r,label='Real Data')
plt.legend(loc=0)
plt.show()

In [32]:
simulation.I[0]

12642.000000000002

In [33]:
RM.Ir[0]

12642

In [34]:
simulation.pE_Ias+simulation.pE_Imi + simulation.pE_Ise + simulation.pE_Icr

1.0

## Acummulated Infected

In [35]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_ac[:index],label='Total Acummulated Infected')
plt.scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Real Data')
plt.legend(loc=0)
plt.show()

## Deaths

In [220]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.B[:index],label='Total Acummulated Deaths')
plt.scatter(RM.Br_tr,RM.Br,label='Real Data')
plt.legend(loc=0)
plt.show()

## UCI/UTI

In [229]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.Hse[:index]+simulation.Hout[:index],label='UCI/UTI Beds')
plt.scatter(RM.sochimi_tr,RM.Hr,label='Real Data')
plt.scatter(RM.sochimi_tr,RM.Hr_tot,label='Real Data')
plt.legend(loc=0)
plt.show()

## VMI

In [222]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.V[:index],label='VMI Usage')
plt.scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
plt.legend(loc=0)
plt.show()

# Objective Functions



In [ ]:
# Date until which we calculate the error (normally when the quarantine dynamic change)
t1_date = datetime(2020,8,15)


t1_day = (t1_date-initdate).days
t1_idx = np.where(np.array(RM.tr)>=t1_day)[0][0]
idx = np.searchsorted(simulation.t,RM.tr[:t1_idx])

## Error:

In [157]:
Err = np.sum(abs(RM.Ir[:t1_idx]-simulation.I[idx]))/np.mean(RM.Ir[:t1_idx])
print(Err)


NameError: name 't1_idx' is not defined

In [ ]:
simulation.peak_date

In [ ]:
RM.initdate